In [72]:
# Nama   : Mia Amaliah 
# NIM    : 1202922002
# Tugas : RFM Analysis


import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/online-retail-ii-uci/online_retail_II.csv


In [73]:
retail = pd.read_csv('../input/online-retail-ii-uci/online_retail_II.csv')
retail.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [74]:
retail.shape

(1067371, 8)

In [75]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [76]:
retail.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [77]:
type_counts = retail['Country'].value_counts()
Country=pd.DataFrame(type_counts)
Country.head()

,Country
United Kingdom,981330
EIRE,17866
Germany,17624
France,14330
Netherlands,5140


In [78]:
retail = retail.dropna()
retail.shape

(824364, 8)

In [79]:
retail=retail[retail['Country']=='Netherlands']
retail.shape

(5140, 8)

In [80]:
df_null = round(100*(retail.isnull().sum())/len(retail), 2)
df_null

Invoice        0.0
StockCode      0.0
Description    0.0
Quantity       0.0
InvoiceDate    0.0
Price          0.0
Customer ID    0.0
Country        0.0
dtype: float64

In [81]:
retail['Customer ID'] = retail['Customer ID'].astype(str)


In [82]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [83]:
max_date = max(retail['InvoiceDate'])
max_date

Timestamp('2011-12-08 12:12:00')

In [84]:
retail['Diff'] = max_date - retail['InvoiceDate']
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Diff
6433,489889,22271,FELTCRAFT DOLL ROSIE,96,2009-12-02 16:52:00,2.55,14646.0,Netherlands,735 days 19:20:00
6434,489889,22078,RIBBON REEL LACE DESIGN,120,2009-12-02 16:52:00,1.85,14646.0,Netherlands,735 days 19:20:00
6435,489889,22082,RIBBON REEL STRIPES DESIGN,120,2009-12-02 16:52:00,1.45,14646.0,Netherlands,735 days 19:20:00
6436,489889,22149,FELTCRAFT 6 FLOWER FRIENDS,120,2009-12-02 16:52:00,2.10,14646.0,Netherlands,735 days 19:20:00
6437,489889,22294,HEART FILIGREE DOVE SMALL,288,2009-12-02 16:52:00,1.06,14646.0,Netherlands,735 days 19:20:00


In [85]:
RFM = retail.groupby('Customer ID')['Diff'].min().reset_index()
RFM.head()

,Customer ID,Diff
0,12382.0,680 days 19:47:00
1,12759.0,50 days 21:23:00
2,12760.0,454 days 03:22:00
3,12761.0,455 days 23:55:00
4,12765.0,470 days 22:21:00


In [86]:
RFM['Diff'] = RFM['Diff'].dt.days
RFM.columns = ['Customer ID','Recency']
RFM.head()

,Customer ID,Recency
0,12382.0,680
1,12759.0,50
2,12760.0,454
3,12761.0,455
4,12765.0,470


In [87]:
RFM_F = retail.groupby('Customer ID')['Invoice'].count().reset_index()
RFM_F.columns = ['Customer ID', 'Frequency']
RFM_F.head()

,Customer ID,Frequency
0,12382.0,1
1,12759.0,302
2,12760.0,21
3,12761.0,27
4,12765.0,68


In [88]:
retail['Amount'] = retail['Quantity']*retail['Price']
RFM_M = retail.groupby('Customer ID')['Price'].sum().reset_index()
RFM_M.head()

,Customer ID,Price
0,12382.0,18.38
1,12759.0,867.58
2,12760.0,62.56
3,12761.0,118.65
4,12765.0,180.33


In [89]:
RFM = RFM.merge(RFM_F,how='inner',on=['Customer ID'])
RFM =RFM.merge(RFM_M,how='inner',on=['Customer ID'])
RFM.head()

,Customer ID,Recency,Frequency,Price
0,12382.0,680,1,18.38
1,12759.0,50,302,867.58
2,12760.0,454,21,62.56
3,12761.0,455,27,118.65
4,12765.0,470,68,180.33
